In [1]:
import urllib
import json
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from keys import client_id, client_secret, app_id

Datasource:
https://docs.google.com/spreadsheets/d/14quQPFErG-hlpsrNgYcX85vW7JMMK5X2vNZrafRcH8c/edit?usp=sharing

In [19]:
data = pd.read_csv('../original_datasets/tableau_time_sereis0403.csv')

In [20]:
states = data['Province_State'].unique().tolist()
dates = data['Date'].unique().tolist()
coords = data.groupby(['Lat', 'Long']).size().index.tolist()
data['avg_tmp'] = np.nan
data['avg_RH'] = np.nan
data['avg_wind'] = np.nan
data['avg_precip'] = np.nan
# data['avg_d_sun'] = np.nan
data['avg_m_tmp'] = np.nan
data['avg_m_RH'] = np.nan
data['avg_m_wind'] = np.nan
data['avg_m_precip'] = np.nan

In [100]:
start = date_range[0].strftime('%m/%d/%Y')
for date in date_range[5:][::5]:
    end = date.strftime('%m/%d/%Y')
    print(start + ' ' + end)
    start = end

01/01/2020 01/06/2020
01/06/2020 01/11/2020
01/11/2020 01/16/2020
01/16/2020 01/21/2020
01/21/2020 01/26/2020
01/26/2020 01/31/2020
01/31/2020 02/05/2020
02/05/2020 02/10/2020
02/10/2020 02/15/2020
02/15/2020 02/20/2020
02/20/2020 02/25/2020
02/25/2020 03/01/2020
03/01/2020 03/06/2020
03/06/2020 03/11/2020
03/11/2020 03/16/2020
03/16/2020 03/21/2020
03/21/2020 03/26/2020
03/26/2020 03/31/2020


In [29]:
# set for up to 30 days a time
# START_TIME = '03/01/2020'
# END_TIME = '04/01/2020'
RADIUS = 350
INTERVAL = 5
q = 0

date_range = pd.date_range('2020-01-01', periods=95, freq='D')
start_time = date_range[0].strftime('%m/%d/%Y')

with open('weather_logs/tableau_log{0}.txt'.format(datetime.now().strftime('%d%m%Y')), 'w') as log:
    msg = 'Start date: {0}\nEnd date: {1}\nInterval:{2}'.format(start_time, date_range[-1]..strftime('%m/%d/%Y'), INTERVAL)
    print(msg)

    for date in date_range[INTERVAL:][::INTERVAL]:
        end_time = date.strftime('%m/%d/%Y')    
        for coord in coords:
            q += 1
            lat = coord[0]
            long = coord[1]
            location = data.loc[data['Lat'] == lat]\
                            .loc[data['Long'] == long]\
                             [['Country_Region','Province_State']]
            country = location.iloc[0][0]
            province = location.iloc[0][1]
            url = 'https://api.aerisapi.com/observations/summary/closest?p={0},{1}&from={2}&to={3}&radius={4}miles&plimit=31&limit=1&'\
                  'client_id={5}&client_secret={6}'.format(lat, long, start_time, end_time, RADIUS, client_id, client_secret)
            msg = 'collecting for {0} location {1}, {2}\n{3}'.format(q, province, country, url)
            print(msg)
            log.write(msg+'\n')
            request = urllib.request.urlopen(url)
            response = request.read()
            json_ = json.loads(response)
            if json_['success']: 
                print('success')
                log.write('success\n')
                try:
                    for day in json_['response'][0]['periods']:
                        date = datetime.strptime(str(day['summary']['ymd']), '%Y%m%d').strftime("%-m/%-d/%Y")
                        print(date)
                        for index in data.loc[data['Date'] == date]\
                                         .loc[data['Lat'] == coord[0]]\
                                         .loc[data['Long'] == coord[1]].index:
                            data.at[index, 'avg_d_tmp'] = day['summary']['temp']['avgC']
                            data.at[index, 'avg_d_RH'] = day['summary']['rh']['avg']
                            data.at[index, 'avg_d_wind'] = day['summary']['wind']['avgKPH']
                            data.at[index, 'avg_d_precip'] = day['summary']['precip']['totalMM']
                except IndexError:
                    if json_['error']:
                        msg = 'No data found for {0}, {1} at {2}\n{3}\r\n'.format(country, province, date, json_['error'])
                        print(msg)
                        log.write(msg)
                except Exception:
                    msg = 'Unknowen error\n{0}\r\n'.format(json_['error'])
                    print(msg)
                    log.write(msg)
                finally:
                    request.close()
    start_time = end_time

collecting for 1 location Tasmania, Australia
https://api.aerisapi.com/observations/summary/closest?p=-41.4545,145.9707&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 2 location nan, New Zealand
https://api.aerisapi.com/observations/summary/closest?p=-40.9006,174.886&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3 location nan, Argentina
https://api.aerisapi.com/observations/summary/closest?p=-38.4161,-63.6167&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 4 location Victoria, Australia
https://api.aerisapi.com/observations/summary/closest?p=-37.8136,144.9631&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tX

collecting for 32 location nan, Angola
https://api.aerisapi.com/observations/summary/closest?p=-11.2027,17.8739&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 33 location nan, Peru
https://api.aerisapi.com/observations/summary/closest?p=-9.19,-75.0152&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 34 location nan, Timor-Leste
https://api.aerisapi.com/observations/summary/closest?p=-8.874217,125.727539&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 35 location nan, Tanzania
https://api.aerisapi.com/observations/summary/closest?p=-6.369,34.8888&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz

collecting for 64 location nan, Sierra Leone
https://api.aerisapi.com/observations/summary/closest?p=8.460555000000001,-11.779889&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 65 location nan, Panama
https://api.aerisapi.com/observations/summary/closest?p=8.538,-80.7821&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 66 location nan, Togo
https://api.aerisapi.com/observations/summary/closest?p=8.6195,0.8248&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 67 location nan, Nigeria
https://api.aerisapi.com/observations/summary/closest?p=9.082,8.6753&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1as

collecting for 95 location nan, Honduras
https://api.aerisapi.com/observations/summary/closest?p=15.2,-86.2419&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 96 location nan, Dominica
https://api.aerisapi.com/observations/summary/closest?p=15.415,-61.371&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 97 location nan, Chad
https://api.aerisapi.com/observations/summary/closest?p=15.4542,18.7322&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 98 location nan, Guatemala
https://api.aerisapi.com/observations/summary/closest?p=15.7835,-90.2308&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&clie

collecting for 126 location Hawaii, US
https://api.aerisapi.com/observations/summary/closest?p=21.17882515,-156.9583592&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 127 location Hawaii, US
https://api.aerisapi.com/observations/summary/closest?p=21.45803166,-157.97121819999998&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 128 location Turks and Caicos Islands, United Kingdom
https://api.aerisapi.com/observations/summary/closest?p=21.694000000000003,-71.7979&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 129 location nan, Burma
https://api.aerisapi.com/observations/summary/closest?p=21.9162,95.956&from=03/01/2020&to=04/01/2020&radius=

...

collecting for 3326 location Washington, US
https://api.aerisapi.com/observations/summary/closest?p=47.62113146,-117.4046494&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3327 location Michigan, US
https://api.aerisapi.com/observations/summary/closest?p=47.63197653,-88.41434263&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3328 location Washington, US
https://api.aerisapi.com/observations/summary/closest?p=47.63479026,-122.6436061&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3329 location Minnesota, US
https://api.aerisapi.com/observations/summary/closest?p=47.64270764,-91.44557998&from=03/01/2020&to=04/01/2020&radius=350miles&plim

collecting for 3356 location North Dakota, US
https://api.aerisapi.com/observations/summary/closest?p=48.22390169,-101.54367629999999&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3357 location Montana, US
https://api.aerisapi.com/observations/summary/closest?p=48.22773388,-112.22527029999999&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3358 location North Dakota, US
https://api.aerisapi.com/observations/summary/closest?p=48.23104458,-100.6324822&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3359 location Minnesota, US
https://api.aerisapi.com/observations/summary/closest?p=48.24539665,-93.78352843&from=03/01/2020&to=04/01/2020&rad

collecting for 3386 location Montana, US
https://api.aerisapi.com/observations/summary/closest?p=48.72085833,-104.50382900000001&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3387 location North Dakota, US
https://api.aerisapi.com/observations/summary/closest?p=48.76623521,-97.5523809&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3388 location Idaho, US
https://api.aerisapi.com/observations/summary/closest?p=48.76684761,-116.4630179&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3389 location Minnesota, US
https://api.aerisapi.com/observations/summary/closest?p=48.77217001,-94.90274472&from=03/01/2020&to=04/01/2020&radius=350miles&pl

collecting for 3416 location Isle of Man, United Kingdom
https://api.aerisapi.com/observations/summary/closest?p=54.2361,-4.5481&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3417 location nan, Lithuania
https://api.aerisapi.com/observations/summary/closest?p=55.1694,23.8813&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3418 location Alaska, US
https://api.aerisapi.com/observations/summary/closest?p=55.32222414,-161.9722021&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3419 location nan, United Kingdom
https://api.aerisapi.com/observations/summary/closest?p=55.3781,-3.4360000000000004&from=03/01/2020&to=04/01/2020&radius=350miles&pl

collecting for 3446 location Alaska, US
https://api.aerisapi.com/observations/summary/closest?p=62.31305045,-149.5741743&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3447 location nan, Sweden
https://api.aerisapi.com/observations/summary/closest?p=63.0,16.0&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3448 location Alaska, US
https://api.aerisapi.com/observations/summary/closest?p=63.67264044,-150.0076108&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
collecting for 3449 location Alaska, US
https://api.aerisapi.com/observations/summary/closest?p=63.87692095,-143.2127643&from=03/01/2020&to=04/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXy

###### HANDLE NaN AND SAVE DATA

In [31]:
data

,Case_Type,Cases,Difference,Date,Country_Region,Province_State,Admin2,Combined_Key,FIPS,Lat,...,avg_wind,avg_precip,avg_m_tmp,avg_m_RH,avg_m_wind,avg_m_precip,avg_d_tmp,avg_d_RH,avg_d_wind,avg_d_precip
0,Deaths,0.0,0.0,2/27/2020,Bahamas,NaN,NaN,NaN,NaN,25.034300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Confirmed,16.0,0.0,2/16/2020,Germany,NaN,NaN,NaN,NaN,51.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Deaths,0.0,0.0,1/31/2020,Canada,Alberta,NaN,NaN,NaN,53.933300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deaths,0.0,0.0,3/4/2020,Australia,Queensland,NaN,NaN,NaN,-28.016700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Confirmed,0.0,0.0,1/25/2020,Suriname,NaN,NaN,NaN,NaN,3.919300,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,88.0,8.7,0.0
5,Deaths,0.0,0.0,2/7/2020,Estonia,NaN,NaN,NaN,NaN,58.595300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Confirmed,0.0,0.0,2/24/2020,Slovakia,NaN,NaN,NaN,NaN,48.669000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Deaths,0.0,0.0,3/23/2020,Timor-Leste,NaN,NaN,NaN,NaN,-8.874217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Deaths,6.0,0.0,2/24/2020,China,Hebei,NaN,NaN,NaN,39.549000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Confirmed,0.0,0.0,2/7/2020,North Macedonia,NaN,NaN,NaN,NaN,41.608600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
backup = data.copy()
# data[['Province_State']] = data[['Province_State']].fillna(0)
# data = data.drop('avg_sun', axis=1)
# data = data.dropna()

In [962]:
##############################################
# COMPUTE AVG PARAMETERS FOR CONFIRMED CASES #
##############################################

conf_data = data[data['Case_Type'] == 'Confirmed']
death_data = data[data['Case_Type'] == 'Deaths']
for coord in coords:
    lat = coord[0]
    long = coord[1]
    indices = conf_data.loc[conf_data['Lat'] == lat].loc[conf_data['Long'] == long].index.tolist()
    avg_tmp = conf_data.loc[conf_data['Lat'] == lat]\
                       .loc[conf_data['Long'] == long]['avg_tmp'].mean()
    
    avg_rh = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_RH'].mean()
    
    avg_wind = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_wind'].mean()
    
    avg_precip = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_precip'].mean()                             
    
    for index in indices:
        conf_data.at[index, 'avg_m_tmp'] = avg_tmp
        conf_data.at[index, 'avg_m_RH'] = avg_rh
        conf_data.at[index, 'avg_m_wind'] = avg_wind
        conf_data.at[index, 'avg_m_precip'] = avg_precip                            

###### COMPUTE AVG PARAMETERS FOR DEATHS CASES

In [963]:
for coord in coords:
    lat = coord[0]
    long = coord[1]
    indices = death_data.loc[data['Lat'] == lat].loc[death_data['Long'] == long].index.tolist()
    avg_tmp = death_data.loc[death_data['Lat'] == lat]\
                       .loc[death_data['Long'] == long]['avg_tmp'].mean()
    
    avg_rh = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_RH'].mean()
    
    avg_wind = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_wind'].mean()
    
    avg_precip = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_precip'].mean()                             
    
    for index in indices:
        death_data.at[index, 'avg_m_tmp'] = avg_tmp
        death_data.at[index, 'avg_m_RH'] = avg_rh
        death_data.at[index, 'avg_m_wind'] = avg_wind
        death_data.at[index, 'avg_m_precip'] = avg_precip                            

In [151]:
#########################
# This was to fix a bug #
#########################

# conf_data = pd.read_csv('../augmented_datasets/tableau_conf_data.csv')
# death_data = pd.read_csv('../augmented_datasets/tableau_death_data.csv')
# conf_data = conf_data.drop('Unnamed: 0', axis=1)
# death_data = death_data.drop('Unnamed: 0', axis=1)
# conf_data['Date'] = conf_data['Date'].apply(lambda x: x.lstrip('0'))
# death_data['Date'] = death_data['Date'].apply(lambda x: x.lstrip('0'))

##############################
# This is for multi-indexing #
##############################

# conf_data['Date'] = pd.DatetimeIndex(conf_data['Date'])
# conf_data.set_index(conf_data['Date'], append=True, inplace=True, drop=True)
# conf_data.loc[2, '2020']

###### COMPUTE GROWTH-FACTOR AND MAX CASES, MAX DATE FOR CONFIRMED CASES

In [183]:
conf_data['max_date'] = ''
conf_data['growth_factor'] = np.nan
conf_data['max_cases'] = np.nan

for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['Cases'].max()
    if not pd.notna(max_cases):
        continue
    max_date = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long][conf_data['Cases'] == max_cases]['Date']
    
    max_date_parsed = datetime.strptime(max_date.values[0], '%m/%d/%Y')
    four_days = max_date_parsed - timedelta(days = 4)
    four_days = four_days.strftime('%-m/%d/%Y')
    
    try:
        four_days_cases = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]\
                         .loc[conf_data['Date'] == four_days]['Cases'].values[0]
        if max_cases > four_days_cases and four_days_cases != 0:
            rate = max_cases / four_days_cases 
            growth_factor = rate**0.25
        else:
            growth_factor = 1
    
    except IndexError:
        growth_factor = 1
        
    indices = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long].index.tolist()
    for index in indices:
        conf_data.at[index, 'max_date'] = max_date.values[0] 
        conf_data.at[index, 'growth_factor'] = growth_factor
        conf_data.at[index, 'max_cases'] = max_cases

/Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [187]:
death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]\
                         .loc[death_data['Date'] == four_days]['Cases']

Series([], Name: Cases, dtype: int64)

###### COMPUTE GROWTH-FACTOR AND MAX CASES, MAX DATE FOR DEATH CASES

In [188]:
death_data['max_date'] = ''
death_data['growth_factor'] = np.nan
death_data['max_cases'] = np.nan

for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]['Cases'].max()
    if not pd.notna(max_cases):
        continue
    max_date = death_data.loc[death_data['Lat'] == lat]\
                        .loc[death_data['Long'] == long][death_data['Cases'] == max_cases]['Date']
    
    max_date_parsed = datetime.strptime(max_date.values[0], '%m/%d/%Y')
    four_days = max_date_parsed - timedelta(days = 4)
    four_days = four_days.strftime('%-m/%d/%Y')
    
    try:
        four_days_cases = death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]\
                         .loc[death_data['Date'] == four_days]['Cases'].values[0]
        if max_cases > four_days_cases and four_days_cases != 0:
            rate = max_cases / four_days_cases 
            growth_factor = rate**0.25
        else:
            growth_factor = 1
    
    except IndexError:
        growth_factor = 1
    
    indices = death_data.loc[death_data['Lat'] == lat]\
                        .loc[death_data['Long'] == long].index.tolist()
    for index in indices:
        death_data.at[index, 'max_date'] = max_date.values[0] 
        death_data.at[index, 'growth_factor'] = growth_factor
        death_data.at[index, 'max_cases'] = max_cases


/Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


24


In [226]:
"""
Somehow there are 580 entries in death_cont that are not in coords
resulting in NaN values for growth factor, we will remove them for the meanwhile
"""
death_data = death_data.dropna()

'\nSomehow there are 580 entries in death_cont that are not in coords\nresulting in NaN values for growth factor, we will remove them for the meanwhile\n'

###### Add 5% interval and averages

In [ ]:
conf_data['5% date'] = ''
conf_data['avg_interval_tmp'] = np.nan
conf_data['avg_interval_RH'] = np.nan

In [ ]:
for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = int(conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['max_cases'].values[0])
    if not pd.notna(max_cases):
        continue
    five_prcnt = int(0.05 * max_cases)
    
    five_prcnt_date = conf_data.loc[conf_data['Lat'] == lat]\
                            .loc[conf_data['Long'] == long][conf_data['Cases'] >= five_prcnt]['Date'].min()
    
    max_date = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['max_date'].values[0]
    
    interval = conf_data.loc[conf_data['Lat'] == lat]\
                            .loc[conf_data['Long'] == long][(conf_data['Date'] >= five_prcnt_date)\
                                                            & (conf_data['Date'] <= max_date)]
    indices = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long].index.tolist()
    for index in indices:
        conf_data.at[index, '5% date'] = five_prcnt_date 
        conf_data.at[index, 'avg_interval_tmp'] = interval['avg_tmp'].mean()
        conf_data.at[index, 'avg_interval_RH'] = interval['avg_RH'].mean()

###### Sanity checks
Manual heuristic comparison of selected samples from the dataset
We compared 5 randomly selected data entries as follows:
1. lat and lan on google map
2. Daily information with Aeris API
3. Daily information with a third party climate source - https://www.worldweatheronline.com/
4. monthly information with a third party data source (This will only be an approximation) https://www.timeanddate.com/weather/israel/tel-aviv/climate

Note that coordinates are in decimal representation

###### Entries for sanity check:
              Lat	Long	avg_tmp	avg_RH	avg_wind	avg_precip	avg_m_tmp	avg_m_RH	avg_m_wind avg_m_precip
- Israel, 03/01/20, 31	35	15.3	68	23	0	14.99310345	74.5862069	13.11034483	0
    * Google maps
        - Checks good at Dimona
    * Check API
        - Checks good at Ben Gurion Air Port
    * Third party daily
        - no Ben gurion only Tel aviv
        - Everything seems good except percipitation
    * Third party monthly
         - Everything seems good except percipitation
- Afghanistan, 03/21/20 33	65	18.7	41	2.4	0	13.71315789	47.07894737	5.684210526	0.855526316
    * Google maps
        - On the mountains 200km south west of Kabul (middle of country)
    * Check API
        - Checks good at kandahar intl ar
    * Third party daily
         - Checks good, wind seems to much
    * Third party monthly
         - tmp ~3c less
         - RH ~10% more
         - m wind 5.6 instead of 14 on third part
         - m precip 0.85 instead of 3.8 in third party
- Queensland Australia, 02/03/20 -28.0167	153.4	25.6	78	12.8	0	24.05	75.55263158	16.66842105	0
    * Google maps
        - southport, in the south eastern corner of Queensland (metropolitan, but not the capital)
    * Check API
        - Checks good at the same town
    * Third party daily
        - Checks good
    * Third party monthly
        - Checks good expect precipitation
- Fiji, 03/06/20 -17.7134	178.065	26.6	90	3.2	0	26.84473684	85.47368421	5.871052632	0
    * Google maps
        - The the northen third of Fiji
    * Check API
        - Checks good for Nausori (60km south east)
    * Third party daily
        - Checks good exepct wind that seems week and precipitation that should be a little
    * Third party monthly
        - seems good exepct for precipitation
- North Dakota, US 03/12/20 47.5289	-99.784	-0.2	73	31.5	0	-4.947368421	80.52631579	18.72368421	0
    * Google maps
        - In the fields around the middle of the state
    * Check API
         - Checks good for harvey (10km north west)
    * Third party daily
        - 
    * Third party monthly
        - checks good except precipitation

After comparing the data we conclude it is relaibale except for precpitation that will be hence forth ignored.

In [35]:
data = pd.read_csv('../augmented_datasets/tableau_conf_data.csv')
RADIUS = 350

coords = [(31, 35), (33, 65), (-28.0167, 153.4), (-17.7134, 178.065), (47.5289, -99.784)] 
dates = ['03/01/2020', '03/21/2020', '03/02/2020', '03/06/2020', '03/12/2020']

for i in range(5):
    lat = coords[i][0]
    long = coords[i][1]
    start_time = dates[i]
    end_time = dates[i]
    location = data.loc[data['Lat'] == lat]\
                    .loc[data['Long'] == long]\
                     [['Country_Region','Province_State']]
    country = location.iloc[0][0]
    province = location.iloc[0][1]
    url = 'https://api.aerisapi.com/observations/summary/closest?p={0},{1}&from={2}&to={3}&radius={4}miles&plimit=31&limit=1&'\
          'client_id={5}&client_secret={6}'.format(lat, long, start_time, end_time, RADIUS, client_id, client_secret)
    print('collecting for {0} location {1}, {2}\n{3}'.format(i, province, country, url))
    request = urllib.request.urlopen(url)
    response = request.read()
    json_ = json.loads(response)
    if json_['success']: 
        print('success')
        try:
            for day in json_['response'][0]['periods']:
                print('Location: {0}, {1}'.format(json_['response'][0]['place']['name'], json_['response'][0]['place']['city']))
                print('avg tmp: {0}'.format(day['summary']['temp']['avgC']))
                print('avg RH: {0}'.format(day['summary']['rh']['avg']))
                print('avg wind: {0}'.format(day['summary']['wind']['avgKPH']))
                print('avg precip: {0}\n'.format(day['summary']['precip']['totalMM']))
        except IndexError:
            if json_['error']:
                msg = 'No data found for {0}, {1} at {2}\n{3}\r\n'.format(country, province, date, json_['error'])
                print(msg)
        except Exception as e:
            raise e
            msg = 'Unknowen error\n{0}\r\n'.format(json_['error'])
            print(msg)
        finally:
            request.close()


collecting for 0 location 0, Israel
https://api.aerisapi.com/observations/summary/closest?p=31,35&from=03/01/2020&to=03/01/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
Location: ben-gurion(civ/m, ben-gurion(civ/m
avg tmp: 15.3
avg RH: 68
avg wind: 23
avg precip: 0

collecting for 1 location 0, Afghanistan
https://api.aerisapi.com/observations/summary/closest?p=33,65&from=03/21/2020&to=03/21/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
Location: kandahar intl ar, kandahar intl ar
avg tmp: 18.7
avg RH: 41
avg wind: 2.4
avg precip: 0

collecting for 2 location Queensland, Australia
https://api.aerisapi.com/observations/summary/closest?p=-28.0167,153.4&from=03/02/2020&to=03/02/2020&radius=350miles&plimit=31&limit=1&client_id=tXyXXyb3Yf492X1asICz5&client_secret=WJi1enKSDGlhO8Nt4Ze00C1F6LEdMpl2O54JIwaM
success
Locati

In [28]:
######################
# SAVE COMPUTED DATA #
######################

data.to_csv('../augmented_datasets/tableau_data0403.csv')
# conf_data.to_csv('../augmented_datasets/tableau_conf_data0403.csv')
# death_data.to_csv('../augmented_datasets/tableau_death_data.csv')